ação.
API: https://www.sptrans.com.br/desenvolvedores
ENDPOINTS:https://www.sptrans.com.br/desenvolvedores/api-do-olho-vivo-guia-de-r
eferencia/documentacao-api/
Utilizar o GTFS da SPTRANS para dados complementares (Dados
Estáticos/Cadastrais).
Enriquecer os dados de paradas com o endereço da localidade, através de latitude e
longitude.
Dados do GTFS: https://gtfs.org/documentation/schedule/reference/


In [18]:
import requests
import pandas as pd
API_TOKEN = "36b9de7cf0f0bcd8cdde826217615c1b5d2e7b71c80397ec50d1ec7620b4f872"
BASE_URL = "http://api.olhovivo.sptrans.com.br/v2.1"
BASE_PATH = ''

In [19]:
session = requests.Session()
def authenticate(api_token):
     # Define the authentication endpoint
    auth_endpoint = f"{BASE_URL}/Login/Autenticar?token={api_token}"
    
    try:
        # Make a POST request to authenticate
        response = session.post(auth_endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False


result = authenticate(API_TOKEN)
print(result.text)

true


In [20]:
def buscar_linha_sentido(session, termosBusca):
     # Define the authentication endpoint
    endpoint = f"{BASE_URL}/Linha/Buscar?termosBusca={termosBusca}"
    
    try:
        # Make a POST request to authenticate
        response = session.get(endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return response
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False


result = buscar_linha_sentido(session, '8000')
print(result.text)



[{"cl":2506,"lc":false,"lt":"8000","sl":1,"tl":1,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":35274,"lc":false,"lt":"8000","sl":2,"tl":1,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":1273,"lc":false,"lt":"8000","sl":1,"tl":10,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":34041,"lc":false,"lt":"8000","sl":2,"tl":10,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"}]


In [21]:
import json

def generic_get(session, endpoint):
     # Define the authentication endpoint
    endpoint = BASE_URL + endpoint    
    print(endpoint)
    try:
        # Make a POST request to authenticate
        response = session.get(endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return response
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False

def kmz(session):
    return generic_get(session, '/KMZ')

def parada_buscar(session, termos_busca):
    return generic_get(session, f"/Parada/Buscar?termosBusca='{termos_busca}'")

def linha_buscar(session, termos_busca):
    return generic_get(session, f"/Linha/Buscar?termosBusca='{termos_busca}'")

def parada_buscar_por_linha(session, codigo_linha):
    return generic_get(session, f"/Parada/BuscarParadasPorLinha?codigoLinha={codigo_linha}")

def previsao_parada(session, codigo_parada):
    return generic_get(session, f'/Previsao/Parada?codigoParada={codigo_parada}')

def previsao_linhas(session, codigo_linha):
    return generic_get(session, f'/Previsao/Linha?codigoLinha={codigo_linha}')

def previsao_parada_all(session):
    return generic_get(session, f'/Previsao')
    
def posicao(session):
    return generic_get(session, f'/Posicao')

def posicao_por_linha(session, codigo_linha):
    return generic_get(session, f"/Posicao/Linha?codigoLinha={codigo_linha}")

# paradas = parada_buscar(session, 'rebouças');
# print(json.dumps(result.json(), indent=4))

# result = parada_buscar_por_linha(session, '1171')
# print(json.dumps(result.json(), indent=4))

# result = previsao_parada(session, '630015011')
# print(json.dumps(result.json(), indent=2))

# result = posicao(session)
# print(json.dumps(result.json(), indent=2))

# result = posicao_por_linha(session, 1171)
# print(json.dumps(result.json(), indent=2))

# result = previsao_parada_all(session)
# print(json.dumps(result.json(), indent=2))

# result = previsao_linhas(session, '1171')
# print(json.dumps(result.json(), indent=2))


paradas = linha_buscar(session, 'USP');
print(json.dumps(result.json(), indent=4))


http://api.olhovivo.sptrans.com.br/v2.1/Linha/Buscar?termosBusca='USP'
[
    {
        "cl": 2506,
        "lc": false,
        "lt": "8000",
        "sl": 1,
        "tl": 1,
        "tp": "P\u00c7A. RAMOS DE AZEVEDO",
        "ts": "TERM. LAPA"
    },
    {
        "cl": 35274,
        "lc": false,
        "lt": "8000",
        "sl": 2,
        "tl": 1,
        "tp": "P\u00c7A. RAMOS DE AZEVEDO",
        "ts": "TERM. LAPA"
    },
    {
        "cl": 1273,
        "lc": false,
        "lt": "8000",
        "sl": 1,
        "tl": 10,
        "tp": "P\u00c7A. RAMOS DE AZEVEDO",
        "ts": "TERM. LAPA"
    },
    {
        "cl": 34041,
        "lc": false,
        "lt": "8000",
        "sl": 2,
        "tl": 10,
        "tp": "P\u00c7A. RAMOS DE AZEVEDO",
        "ts": "TERM. LAPA"
    }
]


In [22]:
!wc posicoes.csv


    9821 1258827767 10437064874 posicoes.csv


In [23]:
import time
import json

with open('posicoes.csv', 'r', encoding='utf-8', errors='replace') as output_file:
    while True:
        try:
            result = result.json()
            linhas = result.get('l',[])
            print(f'{result["hr"]} -> Total de linhas: {len(linhas):,}')
    
            output_file.write(json.dumps(result))
            output_file.write('\n')
            output_file.flush()
            time.sleep(30)
        except Exception as e:
            print(e)
            


    

http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,083
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,083
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,083
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,082
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,081
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,081
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,081
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,081
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,081
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linhas: 2,081
not writable
http://api.olhovivo.sptrans.com.br/v2.1/Posicao
19:33 -> Total de linh

KeyboardInterrupt: 